# House Price Forecasting in Ames, IA
As part of a Kaggle competition, we are going to implement a regression analysis to forecast house prices in Ames, IA.  

## Setting up enviroment
Below we are going to install the necessary packages and download the data needed to build our model

In [15]:
### Import Packages
import numpy as np    #working efficiently with numbers
import pandas as pd    #efficient data structure package
from sklearn.model_selection import ShuffleSplit    #mix our data before splitting into training/testing
from sklearn.metrics import r2_score    #our performance metric for each permutation of GridSearchCV
from sklearn.model_selection import train_test_split    #splitting our data into training/testing sets
from sklearn.preprocessing import Imputer    #assign numeric values to missing values
from sklearn.tree import DecisionTreeRegressor    #building regression from a decision tree
from sklearn.metrics import make_scorer    #make a performance metric
from sklearn.model_selection import GridSearchCV    #assign a dictionary of values that act as a permutation to test
from sklearn.ensemble import AdaBoostRegressor    #ensemble method to overlay on decision tree regressor
import matplotlib.pyplot as plt    #visualize results

## Getting the data for project
Importing the training comma separated values file and parsing out the training features from our target feature

In [2]:
### Import Data
data_train = pd.read_csv('data/train.csv')
print(data_train.head(10))

### Splitting Features and Target Data
prices = data_train['SalePrice']
features = data_train.drop('SalePrice', axis=1)

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   
5   6          50       RL         85.0    14115   Pave   NaN      IR1   
6   7          20       RL         75.0    10084   Pave   NaN      Reg   
7   8          60       RL          NaN    10382   Pave   NaN      IR1   
8   9          50       RM         51.0     6120   Pave   NaN      Reg   
9  10         190       RL         50.0     7420   Pave   NaN      Reg   

  LandContour Utilities    ...     PoolArea PoolQC  Fence MiscFeature MiscVal  \
0         Lvl    AllPub    ...            0    NaN    NaN         NaN       0   
1         Lvl    AllPub

## Preprocessing
Implementing preprocessing steps for our data including dummy variables for categorical data, imputing missing values with averages, and replacing said missing missing values

In [3]:
### Making Dummy Variable
#Removing categorical data for regression analysis
features = pd.get_dummies(features, dummy_na = True)

### Removing NaN Values from DataFrame
fill_NaN = Imputer(missing_values='NaN', strategy='mean', axis=1)

# Create new Data rame with NaN values replaced with the column mean
features2 = pd.DataFrame(fill_NaN.fit_transform(features))
features2.columns = features.columns
features2.index = features.index
print(features2.head(10))

     Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
0   1.0        60.0    65.000000   8450.0          7.0          5.0   
1   2.0        20.0    80.000000   9600.0          6.0          8.0   
2   3.0        60.0    68.000000  11250.0          7.0          5.0   
3   4.0        70.0    60.000000   9550.0          7.0          5.0   
4   5.0        60.0    84.000000  14260.0          8.0          5.0   
5   6.0        50.0    85.000000  14115.0          5.0          5.0   
6   7.0        20.0    75.000000  10084.0          8.0          5.0   
7   8.0        60.0    80.359517  10382.0          7.0          6.0   
8   9.0        50.0    51.000000   6120.0          7.0          5.0   
9  10.0       190.0    50.000000   7420.0          5.0          6.0   

   YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1        ...          \
0     2003.0        2003.0       196.0       706.0        ...           
1     1976.0        1976.0         0.0       978.0        ...           

## Prepping data for modeling
Spliting out data into training and testing sets. We will also cross validate our data using ShuffleSplit

In [38]:
### Splitting into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(features2,
                                                    prices,
                                                    test_size=0.3,
                                                    random_state=0)

### Creating Cross-Validation Sets
cv_sets = ShuffleSplit(n_splits=10,
                       test_size=0.8,
                       random_state=0)

### Building the score function

In [5]:
### Scoring Function
def performance_metric(y_true, y_predict):
    score = r2_score(y_true, y_predict)
    return score

## Building our regression function
We chose to start with a decision tree regression because it is fast to implement and can act as a baseline for future iterations. In addition, we have a GridSearchCV to look over a list of parameters combinations that will optimize our model 

In [6]:
### Decision Tree Regression
tree = DecisionTreeRegressor(random_state=0, presort=True)
params = {'max_depth': range(1, 11),
         'min_samples_leaf': range(1,5),
         'max_features': ['auto','log2']}
scoring_fnc = make_scorer(performance_metric)

### GridSearch for Optimal Model Regular Decision Tree
tree_grid = GridSearchCV(estimator=tree,
                    param_grid=params,
                    scoring=scoring_fnc,
                    cv=cv_sets)
tree_grid_fit = tree_grid.fit(X_train, y_train)
tree_model = DecisionTreeRegressor(random_state=0, **tree_grid_fit.best_params_)
tree_model = tree_model.fit(X_train, y_train)
tree_model

DecisionTreeRegressor(criterion='mse', max_depth=6, max_features='auto',
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=4, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=0,
           splitter='best')

In [7]:
### GridSearch for Optimal Model Regular Decision Tree
AdaTree = AdaBoostRegressor(random_state=0)
params = {'n_estimators': [50,100,200,300],
          'loss': ['linear','square','exponential']}
AdaTree_grid = GridSearchCV(estimator=AdaTree,
                    param_grid=params,
                    scoring=scoring_fnc,
                    cv=cv_sets)
AdaTree_grid_fit = AdaTree_grid.fit(X_train, y_train)
AdaTree_grid_model = AdaBoostRegressor(random_state=0, **AdaTree_grid_fit.best_params_)
AdaTree_grid_model = AdaTree_grid_model.fit(X_train, y_train)
AdaTree_grid_model

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=300, random_state=0)

In [8]:
AdaTree_model = AdaBoostRegressor(tree_model, random_state=0)
AdaTree_model = AdaTree_model.fit(X_train, y_train)
print(AdaTree_model)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=6, max_features='auto',
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=4, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=0,
           splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=50, random_state=0)


In [9]:
AdaTreeCV_model = AdaBoostRegressor(base_estimator=tree_model, random_state=0, **AdaTree_grid_fit.best_params_)
AdaTreeCV_model = AdaTreeCV_model.fit(X_train, y_train)
print(AdaTreeCV_model)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=6, max_features='auto',
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=4, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=0,
           splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=0)


## Making predictions
Using the optimized regression from our GridSearchCV above, we can predict using the test set and measure our performance. This is, again, a baseline for future iterations that we can improve on. 

In [10]:
### Making Predictions with regular regression
tree_pred_train = tree_model.predict(X_train)
tree_perf_train = performance_metric(y_train, tree_pred_train)
print("The r2-score on the training data for the Decision Tree Regressor was", round(tree_perf_train, 4))
tree_pred_test = tree_model.predict(X_test)
tree_perf_test = performance_metric(y_test, tree_pred_test)
print("The r2-score on the testing data for the Decision Tree Regressor was", round(tree_perf_test, 4))

('The r2-score on the training data for the Decision Tree Regressor was', 0.8994)
('The r2-score on the testing data for the Decision Tree Regressor was', 0.7211)


In [63]:
pd.DataFrame(tree_pred_train)

,0
0,227446.233333
1,151807.288288
2,128300.873684
3,109005.961905
4,84730.666667
5,175286.398438
6,109005.961905
7,175286.398438
8,128300.873684
9,122386.135135


In [61]:
plt.figure()
plt.scatter(pd.DataFrame(y_train).index, pd.DataFrame(y_train).iloc[:,0],
            edgecolor="black", c="darkorange", label="data")
plt.plot(pd.DataFrame(y_train).index, tree_pred_test, color="cornflowerblue", label="max_depth=2", linewidth=2)
#plt.plot(X_test, y_2, color="yellowgreen", label="max_depth=5", linewidth=2)
#plt.xlabel("data")
#plt.ylabel("target")
#plt.title("Decision Tree Regression")
#plt.legend()
plt.show()

ValueError: x and y must have same first dimension, but have shapes (1022L,) and (438L,)

In [11]:
### Making Predictions with AdaBoost rergession
AdaTree_grid_pred_train = AdaTree_grid_model.predict(X_train)
AdaTree_grid_perf_train = performance_metric(y_train, AdaTree_grid_pred_train)
print("The r2-score on the training data for the AdaBoost Regression Model was", round(AdaTree_grid_perf_train, 4))
AdaTree_grid_pred_test = AdaTree_grid_model.predict(X_test)
AdaTree_grid_perf_test = performance_metric(y_test, AdaTree_grid_pred_test)
print("The r2-score on the testing data for the AdaBoost Regression Model was", round(AdaTree_grid_perf_test, 4))

('The r2-score on the training data for the AdaBoost Regression Model was', 0.873)
('The r2-score on the testing data for the AdaBoost Regression Model was', 0.7864)


In [12]:
### Making Predictions with AdaBoostRegressor using Grid Searched Decision Tree Regessor as estimator
AdaTree_pred_train = AdaTree_model.predict(X_train)
AdaTree_perf_train = performance_metric(y_train, AdaTree_pred_train)
print("The r2-score on the training data Optimized Decision Tree with AdaBoost Ensemble was",
      round(AdaTree_perf_train, 4))
AdaTree_pred_test = AdaTree_model.predict(X_test)
AdaTree_perf_test = performance_metric(y_test, AdaTree_pred_test)
print("The r2-score on the testing data Optimized Decision Tree with AdaBoost Ensemble was",
      round(AdaTree_perf_test, 4))

('The r2-score on the training data Optimized Decision Tree with AdaBoost Ensemble was', 0.9691)
('The r2-score on the testing data Optimized Decision Tree with AdaBoost Ensemble was', 0.8539)


In [13]:
### Making Predictions with AdaBoostRegressor using Grid Searched Decision Tree Regessor as estimator
AdaTreeCV_pred_train = AdaTreeCV_model.predict(X_train)
AdaTreeCV_perf_train = performance_metric(y_train, AdaTreeCV_pred_train)
print("The r2-score on the training data for the Optimized Decision Tree Regressor overlayed with an Optimized AdaBoost ensemble was",
      round(AdaTreeCV_perf_train, 4))
AdaTreeCV_pred_test = AdaTreeCV_model.predict(X_test)
AdaTreeCV_perf_test = performance_metric(y_test, AdaTreeCV_pred_test)
print("The r2-score on the testing data the Optimized Decision Tree Regressor overlayed with an Optimized AdaBoost ensemble was", round(AdaTreeCV_perf_test, 4))

('The r2-score on the training data for the Optimized Decision Tree Regressor overlayed with an Optimized AdaBoost ensemble was', 0.9732)
('The r2-score on the testing data the Optimized Decision Tree Regressor overlayed with an Optimized AdaBoost ensemble was', 0.858)
